In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import sqlite3
pd.set_option('display.max_colwidth', None)

In [13]:
url = 'https://realfood.tesco.com/search.html?&Course=Lunch&SubType=RECIPE'

In [14]:
html_text=requests.get(url).text

In [15]:
soup = BeautifulSoup(html_text, 'lxml')

In [16]:
info=soup.find_all('div', class_='ddl-search-results__item-details')

### Constructing the first table

In [38]:
j=0
id=1
pg=1
links=[]
df=pd.DataFrame(columns=['Food_id','Name','Rating','Serving Size'])
for counter in range(0,20):
    if pg==1:
        url = 'https://realfood.tesco.com/search.html?&Course=Lunch&SubType=RECIPE'
    else: 
        url= 'https://realfood.tesco.com/search.html?&page='+str(pg)+'&Course=Lunch'
        
    html_text=requests.get(url).text
    soup = BeautifulSoup(html_text, 'lxml')
    info=soup.find_all('div', class_='ddl-search-results__item-details')
    hyper_links=soup.find_all('li',class_='ddl-search-results__item')
    for i in info:
        
            name= i.find('h2',class_='ddl-search-results__item-heading').text
            #print(name)
            try: 
                rating= i.find('div',class_='ddl-search-results__rating').text.strip().split(':')[1]
            except:
                rating='0 out of 5'
                #print('Rating: 0 out of 5')

            try:
                serving_size=i.find('div',class_='ddl-search-results__serves').text
            except:
                serving_size='N/A'
            #print(serving_size,'\n')
            individual_row=[id,name,rating,serving_size]
            df.loc[j]=individual_row
            j=j+1
            id=id+1
            #print(pg)
    for link in hyper_links:
        hyper='https://realfood.tesco.com/'+link.find('a',class_='ddl-search-results__item-link').get('href')
        #print(hyper)
        links.append(hyper)

    pg=pg+1

In [39]:
df

,Food_id,Name,Rating,Serving Size
0,1,Dhal soup,5 out of 5,ServesOverride 4-6
1,2,Garlic crumpets,4 out of 5,Serves 6
2,3,Rosemary and maple gammon,5 out of 5,Serves 8
3,4,Fig and shallot bread with honey-baked Camembert,5 out of 5,Serves 12
4,5,Winter shakshuka,0 out of 5,Serves 6
...,...,...,...,...
595,596,Bacon and potato chowder,5 out of 5,Serves 3
596,597,Rainbow picnic pie,4 out of 5,Serves 12
597,598,Winter vegetable laksa,3 out of 5,Serves 4
598,599,Lentil and apricot tagine,4 out of 5,Serves 1


In [40]:
hyper_links=soup.find_all('li',class_='ddl-search-results__item')

In [41]:
for link in hyper_links:
    hyper='https://realfood.tesco.com/'+link.find('a',class_='ddl-search-results__item-link').get('href')
    #print(hyper)

    links.append(hyper)

In [42]:
new_url='https://realfood.tesco.com//recipes/dhal-soup.html'
html_text_new=requests.get(new_url).text
soup1 = BeautifulSoup(html_text_new, 'lxml')


In [43]:
x2=soup1.find('ul',class_='recipe-detail__meta')

In [44]:
ing=soup1.find_all('li',class_='recipe-detail__list-item')

In [45]:
for i in ing:
    print(i.text)

olive oil
1 tsp black mustard seeds
1 white onion, finely chopped
½ tsp garlic paste
½ tsp ginger paste
2 red chillies, finely chopped
1 tsp baharat
½ tsp ground turmeric
1 tomato, diced
200g red lentils
400ml tin coconut milk
1 tsp salt
Juice of 1 lemon
Crispy fried onions, to serve (optional)


In [46]:
ingredientsss=[element.text for element in ing]

In [47]:
cat=[]
new_url= links[0]
html_text_new=requests.get(new_url).text
soup1 = BeautifulSoup(html_text_new, 'lxml')

In [63]:
j=0 #counter
recepie_id=1
food_id=1
table3=pd.DataFrame(columns=['Recipe_ID','Name','Ingredients'])
table2=pd.DataFrame(columns=['Recepie-id','Food-id','Cal/serving',
                             'Preparation_time','Cooking time','Vegan','Vegetarian',
                             'Egg-free','Dairy-free','Wheat-free','Gluten-free'])
category=['Vegan','Vegetarian','Egg-free','Dairy-free','Wheat-free','Gluten-free']

#links1=links[0:50]
for pi in links:  #pi is just a counter
        cat=[]  #Category_index - 0 or 1
        ing_list=[]  #ingredient_list
        new_url= pi
        html_text_new=requests.get(new_url,timeout=3).text
        soup1 = BeautifulSoup(html_text_new, 'lxml')
        inge=soup1.find_all('li',class_='recipe-detail__list-item')
        types=soup1.find('ul',class_='recipe-detail__meta')
        try:
            name=soup1.find('h1',class_='recipe-detail__headline').text
        except:
            name='No name?'
        try:    
            prep=soup1.find('li',class_='recipe-detail__meta-item recipe-detail__meta-item_time').text.split('and')[0].strip()
        except: 
            prep= 'N/A'
        try:
            cook=soup1.find('li',class_='recipe-detail__meta-item recipe-detail__meta-item_time').text.split('and')[1].strip()
        except: 
            cook='N/A'
        try:   
            calorie= soup1.find('li', class_='recipe-detail__meta-item recipe-detail__meta-item_calories').text
        except: 
            calorie='N/A'
        for type in category:
            if type in str(types): 
                cat.append('1')
            else:
                cat.append('0')
       
        for i in inge:
            ing_list.append(i.text.strip())
            ingredients=str(ing_list).replace('[','').replace(']','').replace("'",'')

        #print(j)
        table2.loc[j]=[recepie_id,food_id,calorie,prep,cook]+cat    
        table3.loc[j]=[recepie_id,name, ingredients]
        j=j+1
        recepie_id=recepie_id+1
        food_id=food_id+1
table2


,Recepie-id,Food-id,Cal/serving,Preparation_time,Cooking time,Vegan,Vegetarian,Egg-free,Dairy-free,Wheat-free,Gluten-free
0,1,1,273 calories / serving,10 mins to prepare,1 hr to cook,1,1,0,1,0,1
1,2,2,283 calories / serving,15 mins to prepare,25 mins to cook,0,1,0,0,0,0
2,3,3,233 calories / serving,10 mins to prepare,"1hr 40 mins to cook, plus resting",0,0,0,1,0,1
3,4,4,507 calories / serving,35 mins to prepare,"40 mins to cook, plus proving",0,0,0,0,0,0
4,5,5,240 calories / serving,10 mins to prepare,30 mins to cook,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
625,626,626,321 calories / serving,30 mins,N/A,0,0,0,0,0,0
626,627,627,455 calories / serving,30 mins to prepare,"1 hr 30 mins to cook, plus chilling",0,1,0,0,0,0
627,628,628,335 calories / serving,15 mins to prepare,35 mins to cook,1,1,0,1,0,0
628,629,629,210 calories / serving,6 mins to prepare,4 mins to cook,0,1,0,1,0,0


### Ingredients table

In [64]:
table3

,Recipe_ID,Name,Ingredients
0,1,Dhal soup recipe,"olive oil, 1 tsp black mustard seeds, 1 white onion, finely chopped, ½ tsp garlic paste, ½ tsp ginger paste, 2 red chillies, finely chopped, 1 tsp baharat, ½ tsp ground turmeric, 1 tomato, diced, 200g red lentils, 400ml tin coconut milk, 1 tsp salt, Juice of 1 lemon, Crispy fried onions, to serve (optional)"
1,2,Garlic crumpets recipe,"6 crumpets, sliced in half, 50g butter, softened, 2 cloves garlic, crushed, 10g flat-leaf parsley, finely chopped, a few leaves reserved for the base, 125g mozzarella, drained and finely chopped, 100g Cheddar, grated"
2,3,Rosemary and maple gammon recipe,"1 small Tesco Finest unsmoked Wiltshire gammon joint, 2 carrots, halved through the middle, 3 cloves, 1 tbsp olive oil, 3 red onions, halved, 50ml maple syrup, pinch of ground cloves, 2 rosemary sprigs, leaves picked and roughly chopped"
3,4,Fig and shallot bread with honey-baked Camembert recipe,"500g strong white bread flour, plus extra for dusting, 7g fast-action dried yeast, 25g butter, 1 tbsp olive oil, 250g shallots, halved, 125g dried figs, chopped, handful basil leaves, 2 x 250g whole Camembert, 1 tbsp clear honey, 1 tbsp pine nuts, 3 thyme sprigs, leaves chopped"
4,5,Winter shakshuka recipe,"2 tbsp olive oil\u200b, 1 large red onion, roughly chopped, 1 green pepper, roughly chopped, 1 small butternut squash, about 700g, peeled and cut into bite-sized pieces, 3 garlic cloves, crushed\u200b, 1 tbsp harissa paste, ¼ tsp ground cumin, 1 tsp smoked paprika\u200b, 2 x 400g tin chopped tomatoes, 1 gluten-free vegetable stock cube, 100g curly kale, stems removed, 6 large eggs, 2 tbsp chopped coriander or parsley"
...,...,...,...
625,626,Bacon and potato chowder recipe,"15g butter, 1 onion, peeled and diced, 4 lean back bacon rashers, chopped, 375g floury potatoes, peeled and diced, 1 chicken stock cube, made up to 175ml, 300ml semi-skimmed milk, 150g frozen sweetcorn, 1 tbsp flat-leaf parsley, chopped"
626,627,Rainbow picnic pie recipe,"700g frozen butternut squash chunks, 2 onions, thinly sliced into half moons, 6 garlic cloves, crushed, 2 tbsp olive oil, plus extra for greasing, 1 tbsp dried thyme, 700g shortcrust pastry, flour, for dusting, 500g spinach, 2 x 250g tubs ricotta, 25g vegetarian hard cheese, grated, 130g tub Tesco Finest basil pesto, 1 lemon, zested, 25g panko breadcrumbs, 460g jar ready-roasted peppers,\xa0drained, patted dry and thickly sliced, 1 egg, beaten"
627,628,Winter vegetable laksa recipe,"2 shallots, roughly chopped, 2 garlic cloves, roughly chopped, 2 stalks lemongrass, finely sliced, 2 red chillies, deseeded and roughly chopped, 5cm fresh ginger, peeled and finely chopped, 1tsp ground turmeric, 4 kaffir lime leaves, roughly chopped, 85g swede, cut into 1cm (½in) cubes, 85g sweet potato, cut into 1cm (½in) cubes, 85g carrot, cut into 1cm (½in) cubes, 4tbsp vegetable oil for frying, 1 x 396g pack tofu, quartered, 1 x 400ml tin coconut milk, 200ml vegetable stock, 4tbsp light soy sauce, 2 tsp sugar, preferably palm if available, 100g (3½oz) green beans cut into 2cm (1in) strips, 150g (5oz) rice noodles, 12 cherry tomatoes, halved, 1 small bunch coriander, leaves picked, ½ bunch mint, leaves picked, 2 spring onions, thinly sliced, 2 limes, halved"
628,629,Lentil and apricot tagine recipe,"3 dried apricots, roughly chopped, 1 shallot, finely diced, 1 tsp harissa paste, ½ tsp ground cumin, 1 tsp red wine vinegar, 1 tsp honey, 8 cherry tomatoes, halved, ½ x 390g tin green lentils, 2 tbsp Greek-style yogurt or tzatziki"


### User's list of desired ingredients

In [28]:
user_req= ['Tomatoe', 'Chicken', 'garlic','onion']

### Finding the food list where it includes the users ingredients


In [29]:
ingredient_list=[]
for i in range(len(user_req)):
    for j in range(len(table3)):
        if user_req[i].lower() in table3['Ingredients'][j]:
            if j in ing_list:
                continue
            ingredient_list.append(j)
table3.loc[ingredient_list]

,Recipe_ID,Food name,Ingredients
8,9,Roasted vegetable tart recipe,"2 large sweet potatoes, skin on, cut into 2cm wedges, 200g cherry tomatoes, 2 red onions, peeled and cut into 1cm wedges, 1 tbsp balsamic vinegar, 2 tbsp olive oil, 35g pumpkin seeds, 125g whole almonds, skin on, 40g unsalted cashew nuts, 100g whole peeled chestnuts, cooked, 75g coconut oil, 1 tbsp pure maple syrup, 8 sprigs rosemary, leaves removed, 60g basil, plus extra leaves to serve, 60g unsalted cashew nuts, roasted, 2 garlic cloves, peeled, 85ml extra-virgin olive oil"
24,25,Sardine fishcakes recipe,"650g floury potatoes, peeled and chopped, 250g pack cherry tomatoes, quartered, ½ small onion, finely chopped, 1 tbsp olive oil, 1 lemon, zested and juiced, 2 x 120g tins sardines in brine, drained, 10g fresh coriander, chopped, 2 tbsp plain flour, 4 tbsp vegetable oil, ½ butterhead lettuce, washed, leaves separated"
26,27,Baked ratatouille with butter bean mash recipe,"2 courgettes, sliced into 1cm rounds, 3 peppers, deseeded and cut into bite-sized chunks, 3 red onions, thickly sliced, 250g salad tomatoes, roughly chopped, 2 tbsp olive oil, 2 tbsp herbes de Provence, 3 garlic cloves, unpeeled, 400g tin chopped tomatoes, 2 x 400g tins butter beans, drained and rinsed, 10g fresh basil"
28,29,Air-fryer firecracker salmon bites recipe,"2 tbsp rice vinegar, 2 tsp reduced-salt soy sauce, 1 tsp sesame oil, 1 tbsp caster sugar\u200b, 1 green finger chilli, very thinly sliced, 1 cucumber, 200g cherry tomatoes, halved or quartered if large, 2 tbsp coriander, chopped, 1 tbsp groundnut oil or sunflower oil, 100g panko breadcrumbs, juice 1 lime, 1 tbsp runny honey, 4 tbsp sweet chilli and garlic sauce, 1 tsp ginger paste, 1 tsp sesame oil, 1 tbsp reduced-salt soy sauce, 2 tbsp sesame seeds, 1 green finger chilli, thinly sliced (optional), 4 boneless salmon fillets, 1 spring onion, sliced, 2 x 250g microwave basmati rice"
38,39,Savoury Chelsea buns recipe,"75g courgette, trimmed and coarsely grated, 1 small carrot (75g), scrubbed and coarsely grated, 450g strong white bread flour, plus 1 tbsp and extra for dusting, 7g sachet fast action yeast, 1 tsp fine salt, 150ml whole milk, 1 medium egg, lightly beaten, 75g unsalted butter, melted, plus extra for greasing, oil, for greasing, 60g green pesto, 75g 30% reduced-fat mature cheese, grated, 4 spring onions, finely sliced\u200b, 75g sundried tomatoes, drained and chopped"
...,...,...,...
578,579,"Smoked haddock, sweet potato and prawn chowder recipe","400g (13oz) undyed smoked haddock fillet, 600ml (1pt) semi-skimmed milk, 2 bay leaves, 6 black peppercorns, 1 tbsp olive oil, plus extra for frying, 40g butter, 1 onion, finely sliced, 2 leeks, thinly sliced, 25g (1oz) plain flour, 400g (13oz) floury potatoes, cut into 2cm (3/4in) cubes, 200g (7oz) sweet potato, cut into 2cm (3/4in) cubes, 4 rashers of streaky bacon, finely chopped, 50g (2oz) cooked cold-water prawns, small bunch chives, snipped, crème fraîche (optional)"
579,580,"Warm bulgur wheat, roast squash and ricotta salad recipe","1kg (2lb) butternut squash, peeled, seeded and cubed, 1 red onion, cut into wedges, 2 tbsp olive oil, 2tsp dried thyme, 125g (4oz) bulgar wheat, 1 head broccoli, cut into florets, 1 red chilli, deseeded and finely chopped, 2tbsp balsamic vinegar, 1 lemon, juiced, 1 tbsp fresh ginger, grated, 100g (3½oz) ricotta cheese"
588,589,Leftover pork and potato hash with poached eggs recipe,"4 potatoes, peeled and cubed, 15ml (1tbsp) sunflower oil, 1/2 onion, finely sliced, 150g (5oz) leftover pork, cubed, tabasco sauce, to taste, Worcestershire sauce, to taste, 4 eggs, poached"
589,590,Mackerel with lemon dressing recipe,"500g (1Ib) new potatoes, halved if large, 30g (1oz) butter, 3 spring onions, sliced, 50g rocket, 8 fillets mackerel, pin bones removed, 4 tbsp plain flour, 2 tbsp groundnut oil, lemon wedges, to serve, 4 tbsp extra-virgin olive oil, 1 lemon, juiced, 2 tbsp capers, rinsed and drained, 3 tbsp chopped basil"


In [19]:
cnn= sqlite3.connect('Food_recipe_database.db')

In [109]:
df.to_sql('Menu',cnn)
table2.to_sql('Description',cnn)
table3.to_sql('Ingredients',cnn)

In [92]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [93]:
%sql sqlite:///food_recipe_database.db

In [94]:
%%sql
select *
from Ingredients

 * sqlite:///food_recipe_database.db
Done.


index,Recipe_ID,Name,Ingredients
0,1,Dhal soup recipe,"olive oil, 1 tsp black mustard seeds, 1 white onion, finely chopped, ½ tsp garlic paste, ½ tsp ginger paste, 2 red chillies, finely chopped, 1 tsp baharat, ½ tsp ground turmeric, 1 tomato, diced, 200g red lentils, 400ml tin coconut milk, 1 tsp salt, Juice of 1 lemon, Crispy fried onions, to serve (optional)"
1,2,Garlic crumpets recipe,"6 crumpets, sliced in half, 50g butter, softened, 2 cloves garlic, crushed, 10g flat-leaf parsley, finely chopped, a few leaves reserved for the base, 125g mozzarella, drained and finely chopped, 100g Cheddar, grated"
2,3,Rosemary and maple gammon recipe,"1 small Tesco Finest unsmoked Wiltshire gammon joint, 2 carrots, halved through the middle, 3 cloves, 1 tbsp olive oil, 3 red onions, halved, 50ml maple syrup, pinch of ground cloves, 2 rosemary sprigs, leaves picked and roughly chopped"
3,4,Fig and shallot bread with honey-baked Camembert recipe,"500g strong white bread flour, plus extra for dusting, 7g fast-action dried yeast, 25g butter, 1 tbsp olive oil, 250g shallots, halved, 125g dried figs, chopped, handful basil leaves, 2 x 250g whole Camembert, 1 tbsp clear honey, 1 tbsp pine nuts, 3 thyme sprigs, leaves chopped"
4,5,Winter shakshuka recipe,"2 tbsp olive oil\u200b, 1 large red onion, roughly chopped, 1 green pepper, roughly chopped, 1 small butternut squash, about 700g, peeled and cut into bite-sized pieces, 3 garlic cloves, crushed\u200b, 1 tbsp harissa paste, ¼ tsp ground cumin, 1 tsp smoked paprika\u200b, 2 x 400g tin chopped tomatoes, 1 gluten-free vegetable stock cube, 100g curly kale, stems removed, 6 large eggs, 2 tbsp chopped coriander or parsley"
5,6,"Baked ham with cranberry glaze, roasted pears and bacon greens recipe","2.6kg gammon joint, 3 small pears, halved, 1 tsp olive oil, 1 shallot, finely chopped, 3 tbsp balsamic vinegar, 200ml port, 200g jar cranberry sauce, 3 tbsp soft brown sugar, 1 tbsp wholegrain mustard, 2 tbsp whole cloves, 300g pack sliced greens, 5 rashers streaky bacon, chopped, 1 tsp olive oil, 2 garlic cloves, sliced, 2 tbsp cider vinegar, 1 tbsp honey, 2 tsp mustard seeds"
6,7,Savoury cheesecake recipe,"400g pack shallots, peeled, 8 garlic cloves, peeled, 2 tbsp olive oil, 150g raw beetroot, trimmed but not peeled, and cut into 1cm wedges, 3 slices gluten-free sliced bread, 70g Tesco Plant Chef Alternative To Grated Hard Cheese, 70g unsalted butter, melted, 300g full fat soft cheese, 300g feta, 250g tub lighter mascarpone, 4 eggs, beaten, 1 lemon, zested, 2 spring onions, finely chopped, 1 tbsp thyme leaves, plus a few extra thyme sprigs, to serve"
7,8,Gluten-free squash and chestnut bake recipe,"4 tbsp olive oil, 1 medium onion, diced, 1 garlic clove, crushed, 800g butternut squash, cut into 2.5cm chunks, 250g pack chestnut mushrooms, 2 tbsp chopped rosemary or thyme leaves, 180g pack cooked and peeled chestnuts, crumbled if large, 250ml dairy-free alternative to cream, 3 tbsp chopped parsley, 25g shelled pistachios, 25g pine nuts, 25g sunflower seeds"
8,9,Turkey Chelsea buns recipe,"320g sheet ready rolled puff pastry, 3 heaped tbsp cranberry sauce, 200g leftover roast veg (see tip below), 100g leftover stuffing, 175g leftover cooked turkey, cut into small pieces, 60g mature Cheddar cheese, coarsely grated, butter, for greasing, fresh thyme leaves, to serve"
9,10,Mushroom nut roast tart with pesto recipe,"500g parsnips, trimmed and peeled, 1 large red onion, cut into 2cm chunks, 2 tbsp maple syrup, 5 tbsp olive oil, plus 1 tsp, 150g pack Portobello mushrooms, sliced, 250g pack chestnut mushrooms,\xa0quartered, 200g pack baby button mushrooms, 50g blanched hazelnuts, 50g blanched almonds, 50g pine nuts, 15g fresh chives, snipped, 15g fresh flat-leaf parsley, 30g pack fresh basil, 2 garlic cloves, crushed, 200g wholemeal spelt flour, plus extra\xa0for dusting, 100g vegetable suet, 80g wholefood 4-seed mix"


In [90]:
%%sql
select *
from Menu m
join Description d
on m.Food_id=d.Food_id

 * sqlite:///food_recipe_database.db
(sqlite3.OperationalError) no such table: Menu
[SQL: select *
from Menu m
join Description d
on m.Food_id=d.Food_id]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [104]:
table2 = table2.rename(columns=({'Food-id': 'Food_id','Recepie-id':'Recipe_id','Cooking time':'Cooking_time'}))

In [106]:
%%sql
select m.Food_id, m.Name,m.Rating, Cooking time
from Menu m
join Description d
on m.Food_id=d.Food_id


 * sqlite:///food_recipe_database.db
(sqlite3.OperationalError) no such column: Cooking
[SQL: select m.Food_id, m.Name,m.Rating, Cooking time
from Menu m
join Description d
on m.Food_id=d.Food_id]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
